<a href="https://colab.research.google.com/github/taharizvi-ai/Chatbot/blob/main/ChatWithMe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_root='/content/drive/My Drive/ChatBot'

Mounted at /content/drive


In [ ]:
import json
import string
import random

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
data_file = open(data_root + '/intents.json').read()
data= json.loads(data_file)

In [ ]:
words =[]
classes = []
dataX = []
dataY = []

for intent in data["intents"]:
  for pattern in intent['patterns']:
    tokens=nltk.word_tokenize(pattern)
    words.extend(tokens)
    dataX.append(pattern)
    dataY.append(intent['tag'])

  if intent['tag'] not in classes:
    classes.append(intent['tag'])

lemmatizer = WordNetLemmatizer()

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
words = sorted(set(words))
classes = sorted(set(classes))

In [ ]:
training = []
out_empty = [0]*len(classes)

for idx, doc in enumerate(dataX):
  bow=[]
  text = lemmatizer.lemmatize(doc.lower())
  for word in words:
    bow.append(1) if word in text else bow.append(0)

    output_row = list(out_empty)
    output_row[classes.index(dataY[idx])] = 1

    training.append([bow, output_row])

random.shuffle(training)
training = np.array(training , dtype=object)
trainX=np.array(list(training[:,0]))
trainy=np.array(list(training[:,1]))


In [ ]:
model = Sequential()
model.add(Dense(128 , input_shape=(len(trainX[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(trainy[0]) , activation = 'softmax'))
adam = tf.keras.optimizers.Adam(learning_rate=0.01 )
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# print(model.summary())
model.fit(x=trainX , y=trainy , epochs=200, verbose=1)

Epoch 1/200
75/75 [==============================] - 1s 4ms/step - loss: 0.8839 - accuracy: 0.7288
Epoch 2/200
75/75 [==============================] - 0s 2ms/step - loss: 0.1369 - accuracy: 0.9487
Epoch 3/200
75/75 [==============================] - 0s 2ms/step - loss: 0.0869 - accuracy: 0.9651
Epoch 4/200
75/75 [==============================] - 0s 3ms/step - loss: 0.0809 - accuracy: 0.9647
Epoch 5/200
75/75 [==============================] - 0s 2ms/step - loss: 0.0709 - accuracy: 0.9630
Epoch 6/200
75/75 [==============================] - 0s 2ms/step - loss: 0.0684 - accuracy: 0.9655
Epoch 7/200
75/75 [==============================] - 0s 2ms/step - loss: 0.0650 - accuracy: 0.9680
Epoch 8/200
75/75 [==============================] - 0s 2ms/step - loss: 0.0654 - accuracy: 0.9664
Epoch 9/200
75/75 [==============================] - 0s 2ms/step - loss: 0.0805 - accuracy: 0.9626
Epoch 10/200
75/75 [==============================] - 0s 3ms/step - loss: 0.0676 - accuracy: 0.9626
Epoch 11/

In [ ]:
def clean_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bagofword(text,vocab):
  tokens=clean_text(text)
  bow=[0]*len(vocab)
  for w in tokens:
    for idx, word in enumerate(vocab):
      if word == w:
        bow[idx] = 1
  return np.array(bow)

def predClass(text , vocab , labels):
  bow = bagofword(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.5
  ypred = [[indx, res] for indx, res in enumerate(result) if res>thresh]
  ypred.sort(key=lambda x: x[1], reverse=True)
  returnlist = []
  for r in ypred:
    returnlist.append(labels[r[0]])
  return returnlist

def getresponse(intentlist, intentsjson):
  if len(intentlist) == 0:
    result = 'Sorry, I dont understand what you mean.'
  else:
    tag = intentlist[0]
    listofintents = intentsjson['intents']
    for i in listofintents:
      if i['tag']==tag:
        result = random.choice(i['responses'])
        break
    return result


In [ ]:
print("Press 0 if you want to exit.")
while True:
  message = input("")
  if message == '0':
    break
  intents = predClass(message, words , classes)
  result = getresponse(intents, data)
  print("Bot: ",result)

Press 0 if you want to exit.
hi
1/1 [==============================] - 0s 22ms/step
Good morning!
how are you
1/1 [==============================] - 0s 19ms/step
Very good and you?
bye
1/1 [==============================] - 0s 20ms/step
Goodbye!
what is your job?
1/1 [==============================] - 0s 19ms/step
I work to serve you as well as possible
what can you do?
1/1 [==============================] - 0s 20ms/step
I can do a lot of things but here are some of my skills, you can ask me: the capital of a country, its currency and its area. A random number. To calculate a math operation.
how are you
1/1 [==============================] - 0s 20ms/step
I am feeling good, you?
i am doing great
1/1 [==============================] - 0s 20ms/step
That is perfect!
0
